In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 31.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2ced857b5bbca16cdc43a1c40f75ab4c57dab65b6bb944486e45e2dc6186fd3a
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
spark

In [7]:
sc = spark.sparkContext


In [8]:
import pandas as pd

In [9]:
df = spark.read.csv('Part1_Crime_data.csv')


In [21]:
from pyspark.sql.types import *

In [66]:
#1 - specifying schema
user_schema = [StructField("X",DoubleType(),True), StructField("Y",DoubleType(),True), StructField("RowID",IntegerType(),True), StructField("CrimeDateTime",StringType(),True), StructField("CrimeCode",StringType(),True), StructField("Location",StringType(),True), StructField("Description",StringType(),True),
 StructField("Inside_Outside",StringType(),True), StructField("Weapon",StringType(),True), StructField("Post",IntegerType(),True), StructField("District",StringType(),True), StructField("Neighborhood",StringType(),True), StructField("Latitude",DoubleType(),True), StructField("Longitude",DoubleType(),True), StructField("GeoLocation",StringType(),True), StructField("Premise",StringType(),True),
 StructField("VRIName",StringType(),True), StructField("Total_Incidents",IntegerType(),True), StructField("Shape",StringType(),True)]


In [67]:
#reading file using schema def
df = spark.read.csv(path='Part1_Crime_data.csv', schema = StructType(user_schema), dateFormat='MM/dd/yyyy',timestampFormat='yyyy-MM-dd HH:mm:ss.SSSSSS')


In [68]:
#removing an invalid top row
df2 = spark.createDataFrame(df.tail(df.count()-1), df.schema)
df2.show()

+----------------+----------------+-----+--------------------+---------+--------------------+--------------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+-----------+------------+---------------+-----+
|               X|               Y|RowID|       CrimeDateTime|CrimeCode|            Location|         Description|Inside_Outside| Weapon|Post| District|        Neighborhood|Latitude|Longitude|       GeoLocation|    Premise|     VRIName|Total_Incidents|Shape|
+----------------+----------------+-----+--------------------+---------+--------------------+--------------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+-----------+------------+---------------+-----+
|1428019.10487147|589532.731060804|    1|2022/03/05 03:43:...|      3NF|       1800 FLEET ST|    ROBBERY - STREET|          null|FIREARM| 213|SOUTHEAST|         FELLS POINT| 39.2847| -76.5913|(39.2847,-76.5913)|       null|

In [38]:
#Cache the DataFrame
df2.cache()

DataFrame[X: double, Y: double, RowID: int, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: int, District: string, Neighborhood: string, Latitude: double, Longitude: double, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: int, Shape: string]

In [40]:
# row count
df2.count()

65533

In [ ]:
#print schema
df2.printSchema() 

In [44]:
#first five rows
df2.head(5)

[Row(X=1428019.10487147, Y=589532.731060804, RowID=1, CrimeDateTime='2022/03/05 03:43:00+00', CrimeCode='3NF', Location='1800 FLEET ST', Description='ROBBERY - STREET', Inside_Outside=None, Weapon='FIREARM', Post=213, District='SOUTHEAST', Neighborhood='FELLS POINT', Latitude=39.2847, Longitude=-76.5913, GeoLocation='(39.2847,-76.5913)', Premise=None, VRIName=None, Total_Incidents=1, Shape=None),
 Row(X=1428019.10487147, Y=589532.731060804, RowID=2, CrimeDateTime='2022/03/05 03:43:00+00', CrimeCode='3NF', Location='1800 FLEET ST', Description='ROBBERY - STREET', Inside_Outside=None, Weapon='FIREARM', Post=213, District='SOUTHEAST', Neighborhood='FELLS POINT', Latitude=39.2847, Longitude=-76.5913, GeoLocation='(39.2847,-76.5913)', Premise=None, VRIName=None, Total_Incidents=1, Shape=None),
 Row(X=1428019.10487147, Y=589532.731060804, RowID=3, CrimeDateTime='2022/03/05 03:43:00+00', CrimeCode='3NF', Location='1800 FLEET ST', Description='ROBBERY - STREET', Inside_Outside=None, Weapon='FI

In [ ]:
#distinct crime codes
df2.select('CrimeCode').distinct().show()

In [ ]:
#Count the number of crimes by the crime codes and order by the resulting counts in descending order
df2.select('CrimeCode').groupby('CrimeCode').count().sort('count', ascending = False).collect()

In [ ]:
# Which neighborhood had the most crimes?
df2.select('Neighborhood').groupby('Neighborhood').count().sort('count', ascending = False).show(1)

In [60]:
from pyspark.sql.functions import *

In [71]:
# Which month of the year had most crimes?
df2.select(('CrimeDateTime')).groupby(month(df2.CrimeDateTime)).count().sort('count', ascending = False).collect()

[Row(month(CrimeDateTime)=None, count=65533)]

In [ ]:
df2.select('Weapon').distinct().show()

In [73]:
df2.select('Weapon').groupby('Weapon').count().sort('count', ascending = True).show(1)

+------+-----+
|Weapon|count|
+------+-----+
|  FIRE|  223|
+------+-----+
only showing top 1 row

